In [ ]:
using DifferentialEquations
#using OrdinaryDiffEq

"""
Decribes the system.

noH is used to solve with H=0 to check for the solution precision
"""
struct parameters
    λ
    ξ
    β
    g
    noH
end
Base.hash(p::parameters, h::UInt) = hash(p.λ,hash(p.ξ,hash(p.β,hash(p.g,h))))

βmin(λ, normalization=2e9) = normalization/(1+4*π/λ)
βmax(λ, normalization=2e9) = normalization/2
genpars(λ, β, noH=false, normalization=2e9) = parameters(λ, sqrt(λ*(normalization-β)), β, sqrt(0.025*4*π), noH)

## System action and derivatives
#'
We start form the action
$$
  S = \int d^4x (K-V)
$$
$$
  K = \frac{(\partial H)^2}{2}+\frac{(\partial X)^2}{2}\cosh^2(\frac{H}{\sqrt{6}})
$$
$$
  V = \frac{36}{4}\left( \lambda \sinh^4(\frac{h}{\sqrt{6}}) +
                  \frac{\beta}{36} \left(1-\exp(-\sqrt{2/3} x) \cosh^2(h/\sqrt{6}) - 6\xi\sinh^2(h/\sqrt{6})\right)^2
                  \right)
$$
Hubble is calculated onthe background as
$$
  H^2 = \frac{K+V}{3}
$$
#'
**Note** Kinetik mixing is present in the equations of motions, but not in the mode expansion.  That is irrelevant because of the small value of background during the generation regime.

In [ ]:
module sym
    using SymEngine
    @vars dh dx h x dδh dδx δh δx λ ξ β
# System definition
    V = 36/4*( λ*sinh(h/sqrt(6))^4 +
                  1/36/β*(1-exp(-sqrt(2/3)*x)*cosh(h/sqrt(6))^2 - 6*ξ*sinh(h/sqrt(6))^2)^2 )
    K = dh^2/2+dx^2/2*cosh(h/sqrt(6))^2
    L = K-V
# ### Assuming diagonal quadratic kinetic term
    dLdh = diff(L,h)
    dLdx = diff(L,x)
    dLddh = diff(L,dh)
    dLddx = diff(L,dx)
    eqnh = (dLdh-diff(dLddh,h)*dh-diff(dLddh,x)*dx)/diff(dLddh,dh)
    eqnx = (dLdx-diff(dLddx,h)*dh-diff(dLddx,x)*dx)/diff(dLddx,dx)
    H = sqrt((K+V)/3)
    Kddhddh = diff(K,dh,dh)
    Kddxddx = diff(K,dx,dx)
    Kddxdh = diff(K,dx,h)
# Much faster? May suffer from "world time problem" - irrelevant!
    lambdify_fast(ex, vars=free_symbols(ex)) = eval(Expr(:function,
                             Expr(:call, gensym(), map(Symbol,vars)...),
                                  convert(Expr, ex)))
    feqnh = lambdify_fast(eqnh,[dh,dx,h,x,λ,ξ,β])
    feqnx = lambdify_fast(eqnx,[dh,dx,h,x,λ,ξ,β])
    feqnδhx = lambdify_fast(diff(eqnh,x),[dh,dx,h,x,λ,ξ,β])
    feqnδhh = lambdify_fast(diff(eqnh,h),[dh,dx,h,x,λ,ξ,β])
    feqnδxx = lambdify_fast(diff(eqnx,x),[dh,dx,h,x,λ,ξ,β])
    feqnδxh = lambdify_fast(diff(eqnx,h),[dh,dx,h,x,λ,ξ,β])
    fH = lambdify_fast(H,[dh,dx,h,x,λ,ξ,β])
    fKh = lambdify_fast(subs(K,dx,0),[dh,dx,h,x,λ,ξ,β])
    fKx = lambdify_fast(subs(K,dh,0),[dh,dx,h,x,λ,ξ,β])
    fV = lambdify_fast(V,[h,x,λ,ξ,β])
    fVx = lambdify_fast(diff(V,x),[h,x,λ,ξ,β])
    fVh = lambdify_fast(diff(V,h),[h,x,λ,ξ,β])
    fVxx = lambdify_fast(diff(V,x,x),[h,x,λ,ξ,β])
    fVhx = lambdify_fast(diff(V,h,x),[h,x,λ,ξ,β])
    fVhh = lambdify_fast(diff(V,h,h),[h,x,λ,ξ,β])
    fVxxx = lambdify_fast(diff(V,x,x,x),[h,x,λ,ξ,β])
    fVhxx = lambdify_fast(diff(V,x,x,h),[h,x,λ,ξ,β])
    fVhhx = lambdify_fast(diff(V,x,h,h),[h,x,λ,ξ,β])
    fVhhh = lambdify_fast(diff(V,h,h,h),[h,x,λ,ξ,β])
end;


fV(h,x,p) = sym.fV(h,x,p.λ,p.ξ,p.β);

FieldType=Array{Float64,1}
fV(u::FieldType, p) = sym.fV(u[3], u[4], p.λ,p.ξ,p.β);
fKh(u::FieldType, p) = sym.fKh(u[1], u[2], u[3], u[4], p.λ,p.ξ,p.β);
fKx(u::FieldType, p) = sym.fKx(u[1], u[2], u[3], u[4], p.λ,p.ξ,p.β);
fH(u::FieldType, p) = sym.fH(u[1], u[2], u[3], u[4], p.λ,p.ξ,p.β);
fV(sol::ODESolution, t) = fV(sol(t), sol.prob.p)
fKh(sol::ODESolution, t) = fKh(sol(t), sol.prob.p)
fKx(sol::ODESolution, t) = fKx(sol(t), sol.prob.p)
fH(sol::ODESolution, t) = fH(sol(t), sol.prob.p);

hminAlt(x,p) = sqrt(6)*asinh(sqrt(
        (1-exp(-sqrt(2.0/3)*x)) / (6*(p.ξ^2+p.λ*p.β)/p.ξ+exp(-sqrt(2.0/3)*x)) ));

function potplot(p; xlim=-0.5:0.05:1, hlim=nothing, vmax=nothing, camera=(60,70))
    hmax = hminAlt(abs(xlim[1]),p)*1.5
    if hlim == nothing
        hlim = LinRange(-hmax,hmax,31)
    end
    if vmax == nothing
        vmax = sqrt(fV(0.0,xlim[end],p)*fV(hlim[1],xlim[end],p))
    end
    plot(hlim, xlim, (x,y)->(v=fV(x,y,p); v>vmax ? vmax : v),
        st=[:surface], zlim=(0,vmax),
        xlabel="\$H\$", ylabel="\$\\Phi\$",
        color=:blues_r,
        camera=camera)
end

Utility function to acces the background

In [ ]:
dH0bg(bgsol,t) = bgsol(t,idxs=1)
dX0bg(bgsol,t) = bgsol(t,idxs=2)
H0bg(bgsol,t) = bgsol(t,idxs=3)
X0bg(bgsol,t) = bgsol(t,idxs=4)
abg(bgsol,t) = exp(bgsol(t,idxs=5))

"""
Calculates the pressure for the background solution
"""
pbg(u::FieldType, p) = fKh(u, p)+fKx(u, p)-sym.fV(u, p)
pbg(bgsol::ODESolution, t) = pbg(bgsol(t), bgsol.prob.p)


"""
Background equations
"""
function bgeqns(du,u,p,t)
    dh,dx,h,x,la = u
    if p.noH
        H = 0
    else
        H = sym.fH(dh,dx,h,x,p.λ,p.ξ,p.β)
    end
    du[1] = -3*H*dh + sym.feqnh(dh,dx,h,x,p.λ,p.ξ,p.β)
    du[2] = -3*H*dx + sym.feqnx(dh,dx,h,x,p.λ,p.ξ,p.β)
    du[3] = dh
    du[4] = dx
    du[5] = H
    nothing
end


function solve_background(p0, tspan=(0.0, 2e6); X0=sqrt(6), N0=0.0, alg=AB5(), dt=1.0)
    fieldu0 = [0.0, 0.0, hminAlt(X0,p0), X0, N0]
    bgprob = ODEProblem(bgeqns, fieldu0, tspan, p0)
    # # Brute forcing another integrator!
    # using Sundials
    # alg=CVODE_BDF()
    # @time bgsol=solve(bgprob, alg, reltol=1e-32,abstol=1e-14);
    solve(bgprob, alg, dt=dt, maxiters=trunc(Int,(tspan[end]-tspan[1])/dt)+1)
end

"""
Solver with generic algorithm and precision!
"""
function solve_background(p0, tspan=(0.0, 2e6); X0=sqrt(6), H0=hminAlt(X0,p0), N0=0.0, alg=AB5(), dt=1.0, reltol=1e-32, abstol=1e-14)
    fieldu0 = [0.0, 0.0, H0, X0, N0]
    bgprob = ODEProblem(bgeqns, fieldu0, tspan, p0)
    solve(bgprob, alg, dt=dt, maxiters=trunc(Int,(tspan[end]-tspan[1])/dt)+1, reltol=reltol, abstol=abstol)
end

"""
This function checks the quality of the background solution.
Note, that the period does not correspond to the normal solution,
as far as the amplitude is larger without the hubble.
"""
function checkBgsolQuality(bgsol; step = 10, dt=nothing, reltol=nothing,abstol=nothing)
    p = bgsol.prob.p
    testp = parameters(p.λ, p.ξ, p.β, p.g, true)
    prob = ODEProblem(bgsol.prob.f.f, bgsol.prob.u0, bgsol.prob.tspan, testp)
    tspan = bgsol.prob.tspan
    if dt == nothing && reltol==nothing
        sol = solve(prob, bgsol.alg, maxiters=trunc(Int,(tspan[end]-tspan[1])/dt)+1)
    elseif dt==nothing
        sol = solve(prob, bgsol.alg, abstol=abstol, reltol=reltol, maxiters=trunc(Int,(tspan[end]-tspan[1])/dt)+1)
    elseif reltol==nothing
        sol = solve(prob, bgsol.alg, dt=dt, maxiters=trunc(Int,(tspan[end]-tspan[1])/dt)+1)
    end
    i = 1:step:length(sol.t)
    scale = 3*fH(sol.u[1],p0)^2
    plot(sol.t[i], [3*fH(u,p0)^2/scale-1 for u in sol.u[i]],
         title="Test of the energy conservation for the ODE solver with H=0",
         xlabel="t", ylabel="\$(E-E_{tot})/E\$")
    # plot()
    # plotenergies(sol)
end

## May want to switch to the interpolator
"Find the first zero crossing of X for the background solution"
t0ind(bgsol)=findfirst(u->u[4]<0, bgsol.u)
t0(bgsol)=bgsol.t[findfirst(u->u[4]<0, bgsol.u)]
t1ind(bgsol)=findnext(u->u[4]>0, bgsol.u, findfirst(u->u[4]<0,bgsol.u))
t1(bgsol)=bgsol.t[findnext(u->u[4]>0, bgsol.u, findfirst(u->u[4]<0,bgsol.u))]
t2(bgsol)=bgsol.t[findnext(u->u[4]<0, bgsol.u, findnext(u->u[4]>0, bgsol.u, findfirst(u->u[4]<0,bgsol.u)))]

function plotenergies!(sol, lab=""; step=10, plotv=false)
    p0 = sol.prob.p
    i=1:step:length(sol.t)
    scale = 1 # 3*fH(sol.u[1],p0)^2
    plot!(sol.t[i],[3*fH(u,p0)^2/scale for u in sol.u[i]],
                    xlabel="\$t\$",
                    label="\$3H^2\$"*lab, lw=1)
    if plotv
        Plots.plot!(sol.t[i],[fV(u,p0)/scale for u in sol.u[i]], ls=:dash, label="\$V\$"*lab, lw=1)
    end
    plot!(sol.t[i],[fKh(u,p0)/scale for u in sol.u[i]], ls=:dot, label="\$K_h\$"*lab, lw=1)
    plot!(sol.t[i],[fKx(u,p0)/scale for u in sol.u[i]], ls=:dashdot, label="\$K_x\$"*lab, lw=1)
    vline!([t0(sol),t1(sol)], label="Starobinsky peak")
end

## Masses
#'
Diagonalization is achieved by
$$
  \left(\begin{array}{c} \phi_L \\ \phi_H \end{array}\right)=
  \left(\begin{array}{cc} \cos\theta & \sin\theta \\
  -\sin\theta & \cos\theta \end{array}\right)
    \left(\begin{array}{c} \delta X \\ \delta H \end{array}\right)
$$
with
$$
  \tan2\theta = \frac{2V_{HX}}{V_{XX}-(V_{HH}-\dot{X}^2/6)}
$$
#'
_Note_ The expressions may be wrong in tachyonic region.  At least, the masses are "exchanged". I am not sure for the mixing angle.  However, as far as this is interpretation only, I would not care for now. It _may_ be important if I would like to get the same creation expresion by replacing the two mode system by just one mode.
#'
For now the noncanonical kinetic term is neglect in the mixing angle calculations. Irrlevant anyway.

In [ ]:
function tan2theta(u::FieldType, p)
    dh, dx, h, x = u
    Vhh = sym.fVhh(h,x,p.λ,p.ξ,p.β)
    Vxx = sym.fVxx(h,x,p.λ,p.ξ,p.β)
    Vhx = sym.fVhx(h,x,p.λ,p.ξ,p.β)
    2*Vhx/(Vxx-Vhh)
end
# This one has wrong ordering of modes for theta>pi/4
theta_alt(u::FieldType, p) = atan(tan2theta(u,p))/2

This version is good for all angles — though it "exchanges" the light and heavy modes in strong tachyonic regime

In [ ]:
"Return Higgs-scalaron mixing for the mode diagonalization"
function theta(u::FieldType, p)
    dh, dx, h, x = u
    Vhh = sym.fVhh(h,x,p.λ,p.ξ,p.β)
    Vxx = sym.fVxx(h,x,p.λ,p.ξ,p.β)
    Vhx = sym.fVhx(h,x,p.λ,p.ξ,p.β)
    atan((Vhh-Vxx-sqrt((Vhh-Vxx)^2+4*Vhx^2))/2/Vhx)
end
theta(bgsol::ODESolution, t) = theta(bgsol(t), bgsol.prob.p)


function thetadot(u::FieldType, p)
    dh, dx, h, x = u
    Vhh = sym.fVhh(h,x,p.λ,p.ξ,p.β)
    Vxx = sym.fVxx(h,x,p.λ,p.ξ,p.β)
    Vhx = sym.fVhx(h,x,p.λ,p.ξ,p.β)
    Vxxx = sym.fVxxx(h,x,p.λ,p.ξ,p.β)
    Vhxx = sym.fVhxx(h,x,p.λ,p.ξ,p.β)
    Vhhx = sym.fVhhx(h,x,p.λ,p.ξ,p.β)
    Vhhh = sym.fVhhh(h,x,p.λ,p.ξ,p.β)
    den = (Vxx-Vhh)^2+4*Vhx^2
    ((Vxx-Vhh)*Vhxx-Vhx*(Vxxx-Vhhx))*dx+((Vxx-Vhh)*Vhhx-Vhx*(Vhxx-Vhhh))*dh
end
thetadot(bgsol::ODESolution,t) = thetadot(bgsol(t), bgsol.prob.p)

The light and heavy masses. Note, that in tachyonic region the negative mode is called light, though it has the higher amplitude.

In [ ]:
function ml2(u::FieldType, p)
    dh, dx, h, x = u
    Vhh = sym.fVhh(h,x,p.λ,p.ξ,p.β)
    Vxx = sym.fVxx(h,x,p.λ,p.ξ,p.β)
    Vhx = sym.fVhx(h,x,p.λ,p.ξ,p.β)
    A = Vhh # - dx^2/6
    B = Vxx
    C = Vhx
    (A+B-sqrt((A-B)^2+4*C^2))/2
#    ( 1/2 * ( (Vhh - dx^2/6) + Vxx )
#          * (1 - sqrt(1 - 4*((Vxx*(Vhh - dx^2/6) - Vhx^2)/(Vhh - dx^2/6 + Vxx)^2))  ) )
end
ml2(bgsol::ODESolution,t) = ml2(bgsol(t), bgsol.prob.p)
wl2(sol::ODESolution,t,k) = ml2(sol,t).+(k./abg(sol,t)).^2

In [ ]:
function mh2(u, p)
    dh, dx, h, x = u
    Vhh = sym.fVhh(h,x,p.λ,p.ξ,p.β)
    Vxx = sym.fVxx(h,x,p.λ,p.ξ,p.β)
    Vhx = sym.fVhx(h,x,p.λ,p.ξ,p.β)
    A = Vhh # - dx^2/6
    B = Vxx
    C = Vhx
    (A+B+sqrt((A-B)^2+4*C^2))/2
#    ( 1/2*((Vhh - dx^2/6) + Vxx)
#         *(1 + sqrt(1 - 4*((Vxx*(Vhh - dx^2/6) - Vhx^2)
#                           /(Vhh - dx^2/6 + Vxx)^2))) )
end
mh2(bgsol::ODESolution, t) = mh2(bgsol(t), bgsol.prob.p)
wh2(sol::ODESolution,t,k) = mh2(sol,t).+(k./abg(sol,t)).^2

In [ ]:
function mh2swapped(u::FieldType, p)
    dh, dx, h, x = u
    Vhh = sym.fVhh(h,x,p.λ,p.ξ,p.β)
    Vxx = sym.fVxx(h,x,p.λ,p.ξ,p.β)
    Vhx = sym.fVhx(h,x,p.λ,p.ξ,p.β)
    A = Vhh # - dx^2/6
    B = Vxx
    C = Vhx
    if abs(Vhh-Vxx-sqrt((Vhh-Vxx)^2+4*Vhx^2)) > abs(2*Vhx)
        (A+B-sqrt((A-B)^2+4*C^2))/2
    else
        (A+B+sqrt((A-B)^2+4*C^2))/2
    end
end
mh2swapped(bgsol::ODESolution, t) = mh2swapped(bgsol(t), bgsol.prob.p)
wh2swapped(bgsol::ODESolution,t,k) = mh2swapped(bgsol,t).+(k/abg(bgsol,t)).^2

W boson masses translated from Anja

Warning: the expression for $m_{W_L}$ is correct only for relatively large $k/a\gg m_T$?

In [ ]:
mwt2(u::FieldType,p) = (p.g*sqrt(6)/2*sinh(u[3]/sqrt(6)))^2
mwt2(bgsol::ODESolution, t) = mwt2(bgsol(t), bgsol.prob.p)
wwt2(sol::ODESolution,t,k) = mwt2(sol,t).+(k./abg(sol,t)).^2

mw2(u, p) = begin
    dh, dx, h, x = u
    V = sym.fV(h,x,p.λ,p.ξ,p.β)
    Vh = sym.fVh(h,x,p.λ,p.ξ,p.β)
    (p.g^2*3/2*sinh(h/sqrt(6))^2 + Vh/tanh(h/sqrt(6))/sqrt(6) - 4*V/6)
end
mw2(bgsol::ODESolution,t) = mw2(bgsol(t), bgsol.prob.p)
ww2(u::FieldType, p, k) = begin
    dh, dx, h, x, la = u
    V = sym.fV(h,x,p.λ,p.ξ,p.β)
    Vh = sym.fVh(h,x,p.λ,p.ξ,p.β)
    (k./exp(la)).^2 .+ ( p.g^2*3/2*sinh(h/sqrt(6))^2 + Vh/tanh(h/sqrt(6))/sqrt(6) - 4*V/6 )
end    
ww2(sol::ODESolution,t,k) = mw2(sol,t).+(k./abg(sol,t)).^2

ww2not(sol::ODESolution,t,k) = mw2(sol,t).-mwt2(sol,t).+(k./abg(sol,t)).^2

ww2exact(u::FieldType, p, k) = begin
    dh, dx, h, x, la = u
    a = exp(la)
    V = sym.fV(h,x,p.λ,p.ξ,p.β)
    Vh = sym.fVh(h,x,p.λ,p.ξ,p.β)
    H = 0 # sym.fH(dh, dx, h, x, p.λ,p.ξ,p.β)
    mt = p.g*sqrt(6)/2*sinh(h/sqrt(6))
    mt2 = mt^2
    dotmt = p.g/2*cosh(h/sqrt(6))*dh
    ( (k/a).^2 .+ mt2
        .+ (k.^2)./((k.^2).+a^2*mt2).*(
            Vh/tanh(h/sqrt(6))/sqrt(6) - 4*V/6
            .+ (3*(dotmt+H*mt)^2)./(((k/a).^2) .+ mt2)
        )
    )
end    
ww2exact(sol::ODESolution,t,k) = ww2exact(sol(t), sol.prob.p, k)

Interesting observation -- seems the dip in the heavy mode depends on the initial conditions quite a lot (Change X0 between 0.5 and sqrt(6))
#' 
## Mode evolution
#' 
Let us make the rescaling of the perturbations as $\phi\to\phi/a^{3/2}$.  Then there are no friction hubble terms, see http://arxiv.org/abs/arXiv:0812.3622

The neglected term in the mass is $\Delta=-\frac{3}{4}(\frac{\dot{a}}{a})^2-\frac{3}{2}\frac{\ddot{a}}{a}=\frac{3}{4}p$

Sign of rotation $\theta$ can be easily checked by changing it -- then both light and heavy modes oscillate a lot during the initial period before the first zero crossing.  So, the current choice is correct.

In [ ]:
"""
    "Exact" perturbation equations.  Hubble terms are neglected.
"""
function perteqns_exact(du,u,par,t)
    kk, bgsol = par
    p = bgsol.prob.p
    ubg = bgsol(t)
    dh,dx,h,x,la = ubg
    a = exp(la)
    eqhh = sym.feqnδhh(dh,dx,h,x,p.λ,p.ξ,p.β)
    eqhx = sym.feqnδhx(dh,dx,h,x,p.λ,p.ξ,p.β)
    eqxh = sym.feqnδxh(dh,dx,h,x,p.λ,p.ξ,p.β)
    eqxx = sym.feqnδxx(dh,dx,h,x,p.λ,p.ξ,p.β)
    Δ = 0 # 3/4*pbg(ubg, p)
    V = sym.fV(h,x,p.λ,p.ξ,p.β)
    Vh = sym.fVh(h,x,p.λ,p.ξ,p.β)
    H = 0 # sym.fH(dh, dx, h, x, p.λ,p.ξ,p.β)
    mt = p.g*sqrt(6)/2*sinh(h/sqrt(6))
    mt2 = mt^2
    dotmt = p.g/2*cosh(h/sqrt(6))*dh    
    for i in eachindex(kk)
        bi = (i-1)*6
        k = kk[i]
        dδh,dδx,δh,δx,dw,w = u[bi+1:bi+6]
        du[bi+1] = eqhh*δh+eqhx*δx-((k/a)^2+Δ)*δh
        du[bi+2] = eqxh*δh+eqxx*δx-((k/a)^2+Δ)*δx
        du[bi+3] = dδh
        du[bi+4] = dδx
        ww2 = ( (k/a)^2 + mt2
                + (k^2)/((k^2)+a^2*mt2)*(
                    Vh/tanh(h/sqrt(6))/sqrt(6) - 4*V/6
                    + (3*(dotmt+H*mt)^2)/((k/a)^2+mt2)
                )
        )
        du[bi+5] = -ww2*w
        du[bi+6] = dw
    end
    nothing
end


"""
    "k/a>>m_T" perturbation equations.  Hubble terms are neglected.
"""
function perteqns(du,u,par,t)
    kk, bgsol = par
    p = bgsol.prob.p
    ubg = bgsol(t)
    dh,dx,h,x,la = ubg
    a = exp(la)
    mmw2 = mw2(ubg,p)
    eqhh = sym.feqnδhh(dh,dx,h,x,p.λ,p.ξ,p.β)
    eqhx = sym.feqnδhx(dh,dx,h,x,p.λ,p.ξ,p.β)
    eqxh = sym.feqnδxh(dh,dx,h,x,p.λ,p.ξ,p.β)
    eqxx = sym.feqnδxx(dh,dx,h,x,p.λ,p.ξ,p.β)
    Δ = 0 # 3/4*pbg(ubg, p)
    for i in eachindex(kk)
        bi = (i-1)*6
        k = kk[i]
        dδh,dδx,δh,δx,dw,w = u[bi+1:bi+6]
        du[bi+1] = eqhh*δh+eqhx*δx-((k/a)^2+Δ)*δh
        du[bi+2] = eqxh*δh+eqxx*δx-((k/a)^2+Δ)*δx
        du[bi+3] = dδh
        du[bi+4] = dδx
        du[bi+5] = -(mmw2+(k/a)^2+Δ)*w
        du[bi+6] = dw
    end
    nothing
end

Vacuum initial conditions

In [ ]:
function pertu_in(pertp0, t; exact=true)
    kk, bgsol = pertp0
    p0 = bgsol.prob.p
    u0 = bgsol(t)
    dh,dx,h,x,la = u0
    a = exp(la)
    θ = theta(u0, p0)
    s = sin(θ)
    c = cos(θ)
    Δ = 0 # 3/4*pbg(ubg, p)
    V = fV(u0,p0)
    Vh = sym.fVh(h,x,p0.λ,p0.ξ,p0.β)
    H = 0 # sym.fH(dh, dx, h, x, p.λ,p.ξ,p.β)
    mt = p0.g*sqrt(6)/2*sinh(h/sqrt(6))
    mt2 = mt^2
    dotmt = p0.g/2*cosh(h/sqrt(6))*dh                        
    uu = zeros(ComplexF64, 6*length(kk))
    for i in eachindex(kk)
        bi = (i-1)*6
        k = kk[i]
        wh = sqrt(mh2(u0,p0)+(k/a)^2)
        wl = sqrt(ml2(u0,p0)+(k/a)^2)
        if exact
            ww2 = ( (k/a)^2 + mt2
                    + (k^2)/((k^2)+a^2*mt2)*(
                        Vh/tanh(h/sqrt(6))/sqrt(6) - 4*V/6
                        + (3*(dotmt+H*mt)^2)/((k/a)^2+mt2)
                    )
                )
        else
            ww2 = mw2(u0,p0)+(k/a)^2
        end
        wwl = sqrt(ww2)
        fh = 1/sqrt(2*wh)
        fl = 1/sqrt(2*wl)
        dfh = -im*sqrt(wh/2)
        dfl = -im*sqrt(wl/2)
        uu[bi+1] = c*dfh+s*dfl
        uu[bi+2] = -s*dfh+c*dfl
        uu[bi+3] = c*fh+s*fl
        uu[bi+4] = -s*fh+c*fl
        uu[bi+5] = -im*sqrt(wwl/2)
        uu[bi+6] = 1/sqrt(2*wwl)
    end
    uu
end

Expand in modes as $[\beta_h,\alpha_h,\beta_l,\alpha_l,\beta_W,\alpha_W]$, with the converntion that $n_k=|\beta|^2$

The choice of phase for negative frequencies is surely wrong!

In [ ]:
function outmodes(pertsol, t; exact=false)
    kk, bgsol = pertsol.prob.p
    p0 = bgsol.prob.p
    u0 = bgsol(t)
    dh,dx,h,x,la = u0
    a = exp(u0[5])
    θ = theta(u0, p0)
    dθ = thetadot(u0, p0)
    s = sin(θ)
    c = cos(θ)
    mmh2 = mh2(u0,p0)
    mml2 = ml2(u0,p0)
    mmw2 = mw2(u0,p0)
    pu = pertsol(t)
    modes = zeros(ComplexF64,8*length(kk))
    V = sym.fV(h,x,p0.λ,p0.ξ,p0.β)
    Vh = sym.fVh(h,x,p0.λ,p0.ξ,p0.β)
    mt = p0.g*sqrt(6)/2*sinh(h/sqrt(6))
    mt2 = mt^2
    H = 0
    dotmt = p0.g/2*cosh(h/sqrt(6))*dh    
    for i in eachindex(kk)
        k = kk[i]
        bi = (i-1)*6
        wh = sqrt(abs(mmh2+(k/a)^2))
        wl = sqrt(abs(mml2+(k/a)^2))
        if exact
            ww2 = ( (k/a)^2 + mt2
                + (k^2)/((k^2)+a^2*mt2)*(
                    Vh/tanh(h/sqrt(6))/sqrt(6) - 4*V/6
                    + (3*(dotmt+H*mt)^2)/((k/a)^2+mt2)
                )
            )
            wwl = sqrt(abs(ww2))
        else
            wwl = sqrt(abs(mmw2+(k/a)^2))
        end
        phiH =  c*pu[bi+3]-s*pu[bi+4]
        phiL =  s*pu[bi+3]+c*pu[bi+4]
        dphiH = c*pu[bi+1]-s*pu[bi+2]-phiL*dθ
        dphiL = s*pu[bi+1]+c*pu[bi+2]+phiH*dθ
        modes[bi+1:bi+6] .= [
            (wh*phiH-im*dphiH)/sqrt(2*wh), (wh*phiH+im*dphiH)/sqrt(2*wh),
            (wl*phiL-im*dphiL)/sqrt(2*wl), (wl*phiL+im*dphiL)/sqrt(2*wl),
            (wwl*pu[bi+6]-im*pu[bi+5])/sqrt(2*wwl), (wwl*pu[bi+6]+im*pu[bi+5])/sqrt(2*wwl)
                            ]
    end
    modes
end

In [ ]:
nH(sol,t) = abs2.(outmodes(sol,t)[1:6:end])
nH(sol,t,ki) = abs2(outmodes(sol,t)[1+(ki-1)*6])
nHbar(sol,t) = abs2.(outmodes(sol,t)[2:6:end])
nHbar(sol,t,ki) = abs2(outmodes(sol,t)[2+(ki-1)*6])
nL(sol,t) = abs2.(outmodes(sol,t)[3:6:end])
nL(sol,t,ki) = abs2(outmodes(sol,t)[3+(ki-1)*6])
nLbar(sol,t) = abs2.(outmodes(sol,t)[4:6:end])
nLbar(sol,t,ki) = abs2(outmodes(sol,t)[4+(ki-1)*6])
nwl(sol,t) = abs2.(outmodes(sol,t)[5:6:end])
nwl(sol,t,ki) = abs2(outmodes(sol,t)[5+(ki-1)*6])
nwlbar(sol,t) = abs2.(outmodes(sol,t)[6:6:end])
nwlbar(sol,t,ki) = abs2(outmodes(sol,t)[6+(ki-1)*6])
nwlexact(sol,t) = abs2.(outmodes(sol,t,exact=true)[5:6:end])
nwlexact(sol,t,ki) = abs2(outmodes(sol,t,exact=true)[5+(ki-1)*6])
nwlbarexact(sol,t) = abs2.(outmodes(sol,t,exact=true)[6:6:end])
nwlbarexact(sol,t,ki) = abs2(outmodes(sol,t,exact=true)[6+(ki-1)*6])


"Get konformal momenta from the current solution"
kk(sol)=sol.prob.p[1]
kk(sol,i)=sol.prob.p[1][i]

In [ ]:
# Convenience functions for current solution
wh2bg(t,ni)=wh2(bgsol,t,kk(sol,ni))
wl2bg(t,ni)=wl2(bgsol,t,kk(sol,ni))
ww2bg(t,ni)=ww2(bgsol,t,kk(sol,ni))
wwt2bg(t,ni)=wwt2(bgsol,t,kk(sol,ni))
ww2notbg(t,ni)=ww2not(bgsol,t,kk(sol,ni))
ww2exact(t,ni)=ww2exact(bgsol,t,kk(sol,ni))

In [ ]:
function rhow(sol,t; exact=false)
    rhow = 0
    modes = outmodes(sol,t, exact=exact)
    bgsol = sol.prob.p[2]
    k = kk(sol)
    if exact
        w2 = ww2exact(bgsol, t, k)
    else
        w2 = ww2(bgsol, t, k)
    end
    for i=1:length(kk(sol))
        rhow += abs2(modes[5+6*(i-1)])*sqrt(abs(w2[i]))*4*pi*k[i]^2/(2*pi*abg(bgsol,t))^3 
    end
    rhow*(k[2]-k[1])
end

In [ ]:
function rhoh(sol,t)
    rho = 0
    modes = outmodes(sol,t)
    bgsol = sol.prob.p[2]
    k = kk(sol)
    w2 = wh2(bgsol, t, k)
    for i=1:length(kk(sol))
        rho += abs2(modes[1+6*(i-1)])*sqrt(abs(w2[i]))*4*pi*k[i]^2/(2*pi*abg(bgsol,t))^3 
    end
    rho*(k[2]-k[1])
end;